In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [2]:
# Load GT detections
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

# Load computed detections
detection_path = 'retinanet_101_detections_trained.pkl'
data_path = '../../AICity_data/train/S03/c010/'

with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

# Load video frames
video_path = '../../AICity_data/train/S03/c010/vdo.avi' #UPDATE PATH WHEN PUSHING
vidcap = cv2.VideoCapture(video_path)
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

## Overlapping boxes approach

In [6]:
def update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    
    # Loop each new detection
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps with max IoU and update if found
        IoUlist = []
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)
            
        indexMax = IoUlist.index(max(IoUlist))
            
        if max(IoUlist) > tolerance and detections_pd.at[indexMax,'updated'] != True:
            detections_pd.at[indexMax,'detection'] = next_detection['detection']
            detections_pd.at[indexMax,'bbox'] = next_detection['bbox']
            detections_pd.at[indexMax,'size'] = next_detection['size']
            detections_pd.at[indexMax,'line'].append(next_detection['line'][0])
            detections_pd.at[indexMax,'updated'] = True
            next_detections_pd.at[index, 'updated'] = True
    


    # #If frame images "imgs" are provided, use optical flow approach to improve tracking
    # if imgs is not None:
    #     idxs = detections_pd.index[detections_pd['updated'] == False].tolist() #Non-updated detections
    #     for ind in idxs:
    #         box = detections_pd.at[ind, 'bbox']
    #         block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]

    #         #Compute optical flow of the not updated tracking bounding box
    #         of = compute_block_of(block, box, imgs[1], 'backward', 10)
            
    #         #update bounding box
    #         if of[0] != 0.0 or of[1] != 0.0:
    #             newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
    #             detections_pd.at[ind, 'bbox'] = newBox
    #             detections_pd.at[ind,'updated'] = True
    #             detections_pd.at[indexMax,'detection'].updateBBox(newBox)
    #             detections_pd.at[indexMax,'line'].append(get_box_center(newBox))



    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    counter = 0
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated']==False]
        new_pd = new_pd.reset_index(drop=True)
        
        # Generate new track number      
        for i in range(len(new_pd)):
            while counter in detections_pd['track'].tolist():
                counter = counter + 1
            new_pd.at[i, 'track'] = counter
            counter = counter + 1

        # Add new tracks
        detections_pd = pd.concat([detections_pd, new_pd])
            
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [7]:
ims = []
detection_history = []

# Get the First Frame
detections_pd = get_detection_dataframe(all_detections['0'])

# Pre-process the first frame if needed
detections_pd = remove_overlaps(detections_pd, 0.9)
detection_history.append(detections_pd)

_, past_frame = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    next_detections_pd = remove_overlaps(next_detections_pd, 0.9)
    detections_pd = update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=[past_frame, image])
    detection_history.append(detections_pd)
    past_frame = image

    for index, row in detections_pd.iterrows():
        image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
    
pkl.dump(detection_history, open('tracking_history.pkl', 'wb'))

100%|██████████| 2139/2139 [03:23<00:00, 10.52it/s]


## Kalman filter approach

In [8]:
# Create Kalman tracker
mot_tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3) # Sort Kalman tracker with default values

results = {}

# Iterate through frames
for frame in tqdm(range(num_frames), desc = "Tracking objects each frame..."):
    dets = []

    _, im = vidcap.read() # Read frame
    detections = get_detection_dataframe(all_detections[str(frame)], iclLineAndUpdate = False).sort_values("track") # Load detections

    # Iterate through frame's detections
    for (track_id, det, bbox, size, colour) in detections.itertuples(index=False):
        dets.append(np.array(bbox))

    trackers = mot_tracker.update(np.array(dets)) # Update tracker with current detections

    for d in trackers: # Store new bboxes
        d = d.astype(np.int32)
        if frame not in results:
            results[frame] = {d[4] % 47: {"bbox": [d[0], d[1], d[2], d[3]]}}
        else:
            results[frame][d[4] % 47] = {"bbox": [d[0], d[1], d[2], d[3]]}

# Save Results to Disk
pkl.dump(results, open("sort_bbox.pkl", "wb"))

Tracking objects each frame...: 100%|██████████| 2141/2141 [00:21<00:00, 100.26it/s]


In [9]:
# Pkl to pd.DataFrame
colours = np.random.rand(47, 3) #Generate Random Colors
old_pkl = pkl.load(open("sort_bbox.pkl", "rb"))
new_pkl = []
df = None

for frame in old_pkl.keys():
    new_data = {"frame": [],"track": [], "bbox": [], "colour": [], "size": [], "detection": []}
    old_data = old_pkl[frame]
    
    for track_id in old_data.keys():
        new_data["track"].append(track_id)
        bbox = old_data[track_id]["bbox"]
        new_data["bbox"].append(bbox)
        new_data["frame"].append("Frame {}".format(frame))
        new_data["colour"].append(np.round(colours[track_id]*255).astype(np.uint8))
        new_data["size"].append(int(np.abs(bbox[0] - bbox[2]) * np.abs(bbox[1] - bbox[3])))
        new_data["detection"].append(VehicleDetection(frame=frame, ID=track_id, width=np.abs(bbox[0] - bbox[2]), conf=0.5,
                                                      height=np.abs(bbox[1] - bbox[3]), left=bbox[0], right=bbox[2], top=bbox[1], bot=bbox[3]))
    
    df = pd.DataFrame.from_dict(new_data).sort_values(by="track")

    new_pkl.append(df)

pkl.dump(new_pkl, open("kalman_tracking.pkl", "wb"))

## Evaluation

In [10]:
total_frames = 2141
initial_frame = 535

In [11]:
detections = {}
with open('tracking_history.pkl','rb') as openFile:
    detections = pkl.load(openFile)

In [12]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    acc, frame_id = tracking_acc(frame, gt_detect, detections, acc)

display_metrics(acc)

100%|██████████| 1605/1605 [01:19<00:00, 20.11it/s]


     num_frames  precision  recall       idp      idr     idf1
acc        1605   0.593536     1.0  0.576804  0.97181  0.72393


In [13]:
detectionsK = {}
with open('kalman_tracking.pkl','rb') as openFile:
    detectionsK = pkl.load(openFile)

In [14]:
total_frames = 2141
initial_frame = 535

accK = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    accK, frame_id = tracking_acc(frame, gt_detect, detectionsK, accK)

display_metrics(accK)

100%|██████████| 1605/1605 [01:06<00:00, 24.01it/s]


     num_frames  precision    recall      idp       idr      idf1
acc        1605   0.708708  0.999627  0.62916  0.887426  0.736303
